In [116]:
import os
import glob
import xarray as xr
import netCDF4 as nc
import numpy as np
from pyproj import Proj, Transformer
import math

In [30]:
current_dir = os.getcwd()

# WFDE5 data
wfde5_path = current_dir + '/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/WFDE5/'
psurf_path = glob.glob(wfde5_path + 'psurf/*.nc')
qair_path = glob.glob(wfde5_path + 'qair/*.nc')
rainf_path = glob.glob(wfde5_path + 'rainf/*.nc')
swdown_path = glob.glob(wfde5_path + 'swdown/*.nc')
tair_path = glob.glob(wfde5_path + 'tair/*.nc')
wind_path = glob.glob(wfde5_path + 'wind/*.nc')
cell_data_path = glob.glob(wfde5_path + 'cell_data/*.nc')

# Testdata
test_path = current_dir + '/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/'
discharge_test_path = test_path + 'discharge.nc'
relhum_test_path = test_path + 'relative_humidity.nc'
precip_test_path = test_path + 'precipitation.nc'
swdown_test_path = test_path + 'radiation.nc'
temp_test_path = test_path + 'temperature.nc'
wind_test_path = test_path + 'wind_speed.nc'
cell_data_test_path = test_path + 'cell_data.nc'

In [32]:
psurf = xr.open_mfdataset(psurf_path)
qair = xr.open_mfdataset(qair_path)
rainf = xr.open_mfdataset(rainf_path)
swdown = xr.open_mfdataset(swdown_path)
tair = xr.open_mfdataset(tair_path)
wind = xr.open_mfdataset(wind_path)
cell_data = xr.open_mfdataset(cell_data_path)

discharge_test = xr.open_mfdataset(discharge_test_path)
relhum_test = xr.open_mfdataset(relhum_test_path)
precip_test = xr.open_mfdataset(precip_test_path)
swdown_test = xr.open_mfdataset(swdown_test_path)
temp_test = xr.open_mfdataset(temp_test_path)
wind_test = xr.open_mfdataset(wind_test_path)
cell_data_test = xr.open_mfdataset(cell_data_test_path)

In [44]:
wind_test

<xarray.Dataset>
Dimensions:      (time: 8760, station: 10)
Coordinates:
  * time         (time) datetime64[ns] 2013-09-01 ... 2014-08-31T23:00:00
    x            (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    y            (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
    z            (station) float64 dask.array<chunksize=(10,), meta=np.ndarray>
Dimensions without coordinates: station
Data variables:
    series_name  (station) object dask.array<chunksize=(10,), meta=np.ndarray>
    crs          int32 ...
    wind_speed   (time, station) float64 dask.array<chunksize=(8760, 10), meta=np.ndarray>

In [82]:
wind

<xarray.Dataset>
Dimensions:  (time: 157800, lon: 88, lat: 47)
Coordinates:
  * time     (time) datetime64[ns] 1999-01-01 ... 2016-12-31T23:00:00
  * lon      (lon) float64 61.25 61.75 62.25 62.75 ... 103.2 103.8 104.2 104.8
  * lat      (lat) float64 16.25 16.75 17.25 17.75 ... 37.75 38.25 38.75 39.25
Data variables:
    Wind     (time, lat, lon) float32 dask.array<chunksize=(744, 47, 88), meta=np.ndarray>
Attributes:
    title:        WATCH Forcing Data methodology applied to ERA5 data
    institution:  Copernicus Climate Change Service
    contact:      http://copernicus-support.ecmwf.int
    comment:      Methodology implementation for ERA5 and dataset production ...
    Conventions:  CF-1.7
    summary:      ERA5 data regridded to half degree regular lat-lon; Genuine...
    reference:    Cucchi et al., 2020, Earth Syst. Sci. Data, 12(3), 2097–212...
    licence:      The dataset is distributed under the Licence to Use Coperni...

In [147]:
# Convert lat and lon to UTM

## Extract lat and lon

lon = wind.lon.values
lat = wind.lat.values

## Making a pyproj object for UTM Zone 41N

crs_4326 = CRS.from_epsg(4326)
crs_32641 = CRS.from_epsg(32641)

## Create transformer to convert from CRS to CRS

transformer = Transformer.from_crs(crs_4326, crs_32641, always_xy = True)

utms = []

for i in range(len(lon)):
    for j in range(len(lat)):
        utms.append(transformer.transform(lon[i], lat[j]))

#transformer.transform(lon, lat)

x_utm = []
y_utm = []

for i in range(len(utms)):
    x_utm.append(utms[i][0])
    y_utm.append(utms[i][1])

In [154]:
np.min(x_utm)

312974.64247951226

In [141]:
utms[0]

(312974.64247951226, 1797388.7268285695)

In [69]:
# Create new netcdf file on Shyft format

fn = test_path + '/test_files/test.nc'
ds = nc.Dataset(fn, 'w', format = 'NETCDF4')

In [70]:
# Add dimensions

time = ds.createDimension('time', None)
x = ds.createDimension('x', 10)
y = ds.createDimension('y', 10)
z = ds.createDimension('z', 10)


In [71]:
times = ds.createVariable('time', 'f4', ('time',))
xs = ds.createVariable('x', 'f4', ('x',))
ys = ds.createVariable('y', 'f4', ('y',))
zs = ds.createVariable('z', 'f4', ('z',))
value = ds.createVariable('value', 'f4', ('time', 'x', 'y', 'z'))
value.units = 'Unknown'

ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(0), x(10), y(10), z(10)
    variables(dimensions): float32 time(time), float32 x(x), float32 y(y), float32 z(z), float32 value(time, x, y, z)
    groups: 

In [80]:
times[:] = wind.time.values
xs[:] = np.arange(40.0, 50.0, 1.0)
ys[:] = np.arange(-110.0, -100.0, 1.0)

In [81]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(157800), x(10), y(10), z(10)
    variables(dimensions): float32 time(time), float32 x(x), float32 y(y), float32 z(z), float32 value(time, x, y, z)
    groups: 

In [65]:
b = xr.open_mfdataset('/Users/jacobqs/Documents/MST Himalaya/MST-Himalaya/shyft_workspace_copy/shyft_workspace/shyft-data/netcdf/orchestration-testdata/test_files/test.nc')
b

<xarray.Dataset>
Dimensions:  ()
Data variables:
    *empty*

In [39]:
ds.cre

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): time(0)
    variables(dimensions): 
    groups: 